In [1]:
author_ids = {
  "Достоевский Ф. М.": 9150,
  "Роллинс Дж.": 59396,
  "Фицджеральд Ф. С.": 28727,
  "Глуховский Д. А.": 53427,
  "Стругацкий А. Н.": 26268,
  "Лукьяненко С. В.": 16626,
  "Фрай М.": 28927,
  "Хантер Э.": 37969,
  "Роулинг Дж. К.": 104832
}

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

In [4]:
from tqdm import tqdm
from multiprocessing import Pool, Lock, Value
from multiprocessing import Manager
from time import sleep
from random import random

mutex = Lock()
n_processed = Value('i', 0)

def get_links_by_author_id(author_id):
    links = []
    for attempt in range(5):
        req = requests.get('https://www.moscowbooks.ru/catalog/author/{}'.format(author_id))
        if req.status_code == 200:
            break
        if req.status_code != 200 and attempt == 4:
            return links
    author_html = req.text
    soup = BeautifulSoup(author_html, 'html.parser')
    pages_attr = soup.find(class_='pager__text', title='перейти на последнюю страницу')
    if pages_attr:
        max_page = int(pages_attr .text)
    else:
        max_page = 1
    for page_id in range(1, max_page + 1):
        author_page_html = requests.get('https://www.moscowbooks.ru/catalog/author/{}/?page={}'.format(author_id, page_id)).text
        soup = BeautifulSoup(author_page_html, 'html.parser')

        page_ids = soup.find_all(class_='book-preview__fav fav js-fav')
        page_ids = [book_id['data-productid'] for book_id in page_ids]
        
        links.extend(page_ids)
        sleep(0.5 + random() / 2)
    return links
def get_links_by_author_id_wrapper(uid):
    res = get_links_by_author_id(uid) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        print(f"\r{n_processed.value} objects are processed..., author_id={uid}", end='', flush=True)
    return res

In [5]:
from multiprocessing import Pool, Lock, Value
with Pool(processes=5) as pool:
    res = pool.map(get_links_by_author_id_wrapper, author_ids.values())

9 objects are processed..., author_id=379692

In [6]:
import functools
import operator
book_links = functools.reduce(operator.iconcat, res, [])
len(book_links)

243

In [7]:
import re
from time import sleep
from random import random
import unicodedata

RE_SPACES_REMOVE = re.compile(r'\s+') #3
RE_BREAK_REMOVE = re.compile(r'\r\n')
RE_FIX_PRICE = re.compile('(?<=\d)+ (?=\d)+')

def extract_book_info(book_id, max_attempts=5): #2
    details = dict() #7
    
    for attempt in range(max_attempts):
        req = requests.get('https://www.moscowbooks.ru/book/{}'.format(book_id))
        if req.status_code == 200:
            break
        if req.status_code != 200 and attempt == max_attempts - 1:
            return details
    book_html = req.text
    book_html = RE_BREAK_REMOVE.sub('', book_html)
    book_html = RE_SPACES_REMOVE.sub(' ', book_html).strip()
    soup = BeautifulSoup(book_html, 'html.parser')
    
    author_name = ', '.join([a.text.strip() for a in soup.find(class_='page-header__author')\
                             .find_all('a')]) #5, 6
    details['Автор'] = author_name

    details['Название'] = soup.find('span', class_='link-gray-light').text.strip()
    
    soup = soup.find('div', class_='book') #8
    
    relative_path = soup.find(class_='book__img book__img_default gallery__img')['src']
    img_path = 'https://www.moscowbooks.ru{}'.format(relative_path)
    details['Обложка'] = img_path
    
    stickers = soup.find(class_='book__stickers stickers stickers_lg')
    if stickers:
        gift_stickers = stickers.find(class_='stickers__icon icon-gift')
        gift_stickers = [] if not gift_stickers else [gift_stickers['title']]
        stickers = ', '.join(stickers.text.split() + gift_stickers)
    details['Стикеры'] = stickers
    
    details['Наличие'] = bool(soup.find(class_='book__shop-details').find(class_='icon-check'))
    
    price = soup.find(class_='book__price')
    if price:
        price = price.text.strip()
        price = unicodedata.normalize('NFKD', price)
        price = RE_FIX_PRICE.sub('', price)
    details['Цена'] = price
    
    genres = soup.find_all(class_='genre_link', text=True)
    if genres:
        genres = ', '.join([g.text.strip() for g in genres]) #5
    else:
        genres = ''
    details['Жанры'] = genres
    
    details['Рейтинг'] = int(soup.find(class_='book___rating-stars rating-stars rating-stars_lg')['data-rate'])

    cells = soup.find_all(class_='book__details-item')
    for cell in cells:
        book_details_name = cell.dt.text.replace(':', '').strip()
        book_deatils_value = cell.find(class_='book__details-value').text.strip()
        details[book_details_name] = book_deatils_value
    details['Код товара'] = int(details['Код товара'])
    
    #4 Честно говоря, сначала искал все теги 'book__decription', но тогда приходилось избавляться от
    # 'Читать далее...' и 'Аннотация к книге...', удаляя теги 'b' и 'a', что в итоге раздувало код,
    # и прошлое решение мне показалось оптимальным
    # Другого решения не вижу..(
    
    descr = soup.find_all(class_='book__description')[-1]
    ann = descr.find('b') #Удаляем 'Аннотация...'
    if ann:
        ann.decompose()
    read_more = descr.find('a')#Удаляем 'Читать далее...'
    if read_more:
        read_more.decompose()
    if descr:
        details['Описание'] = descr.text
    
    #Случайная задержка чтобы так сразу не вычислили
    sleep(1.5 + random() / 2)
    return details


In [8]:
test = extract_book_info('726985')

In [9]:
test

{'Автор': 'Роулинг Дж. К.',
 'Название': 'Гарри Поттер и Философский камень',
 'Обложка': 'https://www.moscowbooks.ru/image/book/454/w259/i454685.jpg',
 'Стикеры': '',
 'Наличие': True,
 'Цена': '487 руб.',
 'Жанры': '',
 'Рейтинг': 1,
 'Издательство': 'Махаон; Азбука-Аттикус',
 'Год издания': '2014',
 'Место издания': 'Москва',
 'Возраст': '11 +',
 'Язык текста': 'русский',
 'Язык оригинала': 'английский',
 'Перевод': 'Спивак М.',
 'Тип обложки': 'Твердый переплет',
 'Формат': '84х108 1/32',
 'Размеры в мм (ДхШхВ)': '200x130',
 'Вес': '475 гр.',
 'Страниц': '432',
 'Тираж': '35000 экз.',
 'Код товара': 726985,
 'Артикул': 'А0000007533',
 'ISBN': '978-5-389-07435-4',
 'В продаже с': '21.02.2014',
 'Описание': 'Важные отличия нового издания книг о Гарри Поттере от предыдущего — знаменитый перевод Марии Спивак и новое современное оформление. Некоторое время назад в одном из интервью Мария Спивак очень точно объяснила, чем именно отличается ее перевод от ранее издававшегося:«Известно, что

In [10]:
multiple_author_check = extract_book_info('855979')
multiple_author_check['Автор']

'Баева Н. Д., Зебряк Т. А.'

In [11]:
rating_check_1 = extract_book_info('726985') 
rating_check_4 = extract_book_info('995166')
rating_check_5 = extract_book_info('822220')
rating_check_1['Рейтинг'], rating_check_4['Рейтинг'], rating_check_5['Рейтинг']

(1, 4, 5)

In [12]:
exist_check_true = extract_book_info('1017467')
exist_check_false = extract_book_info('1004913')
exist_check_true['Наличие'], exist_check_false['Наличие']

(True, False)

In [13]:
stickers_check_new = extract_book_info('1017583')
stickers_check_bestseller = extract_book_info('1011825')
stickers_check_both = extract_book_info('1016150')
stickers_check_gift = extract_book_info('873080')
stickers_check_new['Стикеры'], stickers_check_bestseller['Стикеры'], stickers_check_both['Стикеры'], stickers_check_gift['Стикеры']

('Новинка', 'Бестселлер', 'Новинка, Бестселлер', 'Идея подарка')

In [14]:
check_genres_1 = extract_book_info('1013153')
check_genres_2 = extract_book_info('1000435')
check_genres_3 = extract_book_info('1013396')
check_genres_1['Жанры'], check_genres_2['Жанры'], check_genres_3['Жанры']

('Современная зарубежная проза',
 'Классическая российская проза',
 'Триллер, Современный детектив')

In [15]:
check_price = extract_book_info('1013659')
check_price_spaces = extract_book_info('902567')
check_price['Цена'], check_price_spaces['Цена']

('614 руб.', '2888 руб.')

In [16]:
mutex = Lock()
n_processed = Value('i', 0)

def extract_book_info_wrapper(book_id):
    res = extract_book_info(book_id) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

In [17]:
with Pool(processes=10) as pool:
    result = pool.map(extract_book_info_wrapper, book_links)

240 objects are processed...

In [18]:
import pandas as pd
df = pd.DataFrame(result)
df.head()

,Автор,Название,Обложка,Стикеры,Наличие,Цена,Жанры,Рейтинг,Издательство,Год издания,...,Бумага,Обрез,Иллюстрации,Язык оригинала,Перевод,Футляр,Производитель,Год производства,Место производства,Иллюстраторы
0,Достоевский Ф. М.,Преступление и наказание,https://www.moscowbooks.ru/image/book/675/w259...,,True,136 руб.,Классическая российская проза,5,Эксмо,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Достоевский Ф. М.,Игрок,https://www.moscowbooks.ru/image/book/668/w259...,,False,220 руб.,,0,Искателькнига,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Достоевский Ф. М.,Чужая жена и муж под кроватью,https://www.moscowbooks.ru/image/book/664/w259...,,True,144 руб.,Классическая российская проза,0,АСТ,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Достоевский Ф. М.,Полное собрание романов в двух томах. В 2 книгах,https://www.moscowbooks.ru/image/book/661/w259...,,True,1880 руб.,Классическая российская проза,0,АЛЬФА-КНИГА,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Достоевский Ф. М.,Преступление и наказание,https://www.moscowbooks.ru/image/book/659/w259...,None,True,12654 руб.,,0,NaN,2019,...,Офсетная,Рисованный,С иллюстрациями,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.sort_values(by=['Код товара'], inplace=True)
df.fillna(value ='', inplace=True)
df['Артикул'] = df['Артикул'].astype(str)
df.head()

,Автор,Название,Обложка,Стикеры,Наличие,Цена,Жанры,Рейтинг,Издательство,Год издания,...,Бумага,Обрез,Иллюстрации,Язык оригинала,Перевод,Футляр,Производитель,Год производства,Место производства,Иллюстраторы
77,Достоевский Ф. М.,Преступление и наказание. 2 CD: mp3,https://www.moscowbooks.ru/image/book/141/w259...,,True,240 руб.,,0,,,...,,,,,,,Ардис,2005,Москва,
76,Достоевский Ф. М.,Собрание сочинений: В 4 томах,https://www.moscowbooks.ru/image/book/245/w259...,,True,5912 руб.,Классическая российская проза,0,Янтарный сказ,2008,...,,,,,,,,,,
75,Достоевский Ф. М.,Идиот,https://www.moscowbooks.ru/image/book/257/w259...,,True,420 руб.,Классическая российская проза,0,Эксмо,2009,...,,,,,,,,,,
74,Достоевский Ф. М.,Преступление и наказание. В 2 томах,https://www.moscowbooks.ru/image/book/268/w259...,,True,8593 руб.,,0,,2008,...,,,,,,,,,,
73,Достоевский Ф. М.,Идиот. Бедные люди. 1 CD: mp3,https://www.moscowbooks.ru/image/book/285/w259...,,True,192 руб.,,0,,,...,,,,,,,Ардис,2009,Москва,


In [29]:
with open('hw_3_fixed.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False, escapechar=';')